# Event Splitter Inference

Run event-level edge-affinity inference and save per-event predictions.
Output files keep one row per input event, with edge lists and affinity scores.


In [1]:
from pathlib import Path

import pyarrow.parquet as pq

from pioneerml.common.zenml import load_step_output
from pioneerml.common.zenml import utils as zenml_utils
from pioneerml.pipelines.inference.event_splitting import event_splitting_inference_pipeline

PROJECT_ROOT = zenml_utils.find_project_root()
zenml_utils.setup_zenml_for_notebook(root_path=PROJECT_ROOT, use_in_memory=True)


Using ZenML repository root: /workspace
Ensure this is the top-level of your repo (.zen must live here).


In [2]:
# Inputs

def _pick_pred(pred_dir: Path, main_path: Path) -> Path | None:
    primary = pred_dir / f"{main_path.stem}_preds.parquet"
    if primary.exists():
        return primary
    latest = pred_dir / f"{main_path.stem}_preds_latest.parquet"
    if latest.exists():
        return latest
    return None

main_dir = Path(PROJECT_ROOT) / "data"
main_paths = sorted(main_dir.glob("ml_output_*.parquet"))

# Example: uncomment to use fewer files
main_paths = main_paths[:1]

group_classifier_dir = Path(PROJECT_ROOT) / "data" / "group_classifier"
group_splitter_dir = Path(PROJECT_ROOT) / "data" / "group_splitter"
endpoint_dir = Path(PROJECT_ROOT) / "data" / "endpoint_regressor"

paired = []
for mp in main_paths:
    gc = _pick_pred(group_classifier_dir, mp)
    gs = _pick_pred(group_splitter_dir, mp)
    ep = _pick_pred(endpoint_dir, mp)
    if gc is not None and gs is not None and ep is not None:
        paired.append((str(mp.resolve()), str(gc.resolve()), str(gs.resolve()), str(ep.resolve())))

if not paired:
    raise RuntimeError(
        "No aligned event-splitting inference inputs found. "
        "Run upstream inference pipelines first."
    )

parquet_paths = [p[0] for p in paired]
group_probs_parquet_paths = [p[1] for p in paired]
group_splitter_parquet_paths = [p[2] for p in paired]
endpoint_parquet_paths = [p[3] for p in paired]

model_path = None  # default: latest trained model in trained_models/event_splitter
output_dir = str((Path(PROJECT_ROOT) / "data" / "event_splitter").resolve())

print(f"Input files: {len(parquet_paths)}")


Input files: 1


In [3]:
# Run inference pipeline
run = event_splitting_inference_pipeline.with_options(enable_cache=False)(
    parquet_paths=parquet_paths,
    group_probs_parquet_paths=group_probs_parquet_paths,
    group_splitter_parquet_paths=group_splitter_parquet_paths,
    endpoint_parquet_paths=endpoint_parquet_paths,
    model_path=model_path,
    output_dir=output_dir,
    pipeline_config={
        "loader": {
            "config_json": {
                "mode": "inference",
                "batch_size": 16,
                "chunk_row_groups": 4,
                "chunk_workers": 0,
                "use_group_probs": True,
                "use_splitter_probs": True,
                "use_endpoint_preds": True,
            }
        },
        "inference": {"threshold": 0.5},
        "save_predictions": {"check_accuracy": False, "write_timestamped": False},
    },
)

export_info = load_step_output(run, "save_event_splitter_predictions")
print(export_info)


Initiating a new run for the pipeline: event_splitting_inference_pipeline.
Caching is disabled by default for event_splitting_inference_pipeline.
Using user: default
Using stack: default
  deployer: default
  artifact_store: default
  orchestrator: default
You can visualize your pipeline runs in the ZenML Dashboard. In order to try it locally, please run zenml login --local.
Step load_event_splitter_inference_inputs has started.
Step load_event_splitter_inference_inputs has finished in 0.506s.
Step load_event_splitter_model has started.
Step load_event_splitter_model has finished in 0.118s.
Step run_event_splitter_inference has started.
Step run_event_splitter_inference has finished in 5.957s.
Step save_event_splitter_predictions has started.
Step save_event_splitter_predictions has finished in 0.384s.
Pipeline run has finished in 9.672s.
{'predictions_path': '/workspace/data/event_splitter/ml_output_000_preds.parquet', 'predictions_paths': ['/workspace/data/event_splitter/ml_output_00

In [4]:
# Inspect export outputs
predictions_paths = [Path(p) for p in (export_info.get("predictions_paths") or [])]
if not predictions_paths and export_info.get("predictions_path"):
    predictions_paths = [Path(export_info["predictions_path"])]
metrics_path = Path(export_info["metrics_path"])

print("predictions_paths:")
for p in predictions_paths:
    print(" ", p)
print("metrics:", metrics_path)
print(metrics_path.read_text())


predictions_paths:
  /workspace/data/event_splitter/ml_output_000_preds.parquet
metrics: /workspace/data/event_splitter/metrics_latest.json
{
  "accuracy": null,
  "confusion": null,
  "exact_match": null,
  "loss": null,
  "mean_logit": 0.17631614003234883,
  "mode": "event_splitter",
  "model_path": "/workspace/trained_models/event_splitter/event_splitter_20260219_021339_torchscript.pt",
  "output_path": "/workspace/data/event_splitter/ml_output_000_preds.parquet",
  "output_paths": [
    "/workspace/data/event_splitter/ml_output_000_preds.parquet"
  ],
  "threshold": 0.5,
  "validated_endpoint_files": [
    "/workspace/data/endpoint_regressor/ml_output_000_preds.parquet"
  ],
  "validated_files": [
    "/workspace/data/ml_output_000.parquet"
  ],
  "validated_group_probs_files": [
    "/workspace/data/group_classifier/ml_output_000_preds.parquet"
  ],
  "validated_group_splitter_files": [
    "/workspace/data/group_splitter/ml_output_000_preds.parquet"
  ]
}


In [5]:
# Optional: verify parquet schema + small sample (avoids loading full file)
import gc
import pyarrow as pa
import pyarrow.parquet as pq

if not predictions_paths:
    raise RuntimeError("No prediction parquet files were exported.")

pf = pq.ParquetFile(predictions_paths[0])
print("file:", predictions_paths[0])
print("rows:", pf.metadata.num_rows)
print(pf.schema_arrow)

if pf.num_row_groups > 0:
    sample = pf.read_row_group(0).slice(0, 3)
    print(sample)
else:
    sample = None
    print("No row groups found.")

# Release notebook-held references after inspection
del sample, pf
gc.collect()
pa.default_memory_pool().release_unused()


file: /workspace/data/event_splitter/ml_output_000_preds.parquet
rows: 1024
event_id: int64
time_group_ids: list<element: int64>
  child 0, element: int64
edge_src_index: list<element: int64>
  child 0, element: int64
edge_dst_index: list<element: int64>
  child 0, element: int64
pred_edge_affinity: list<element: float>
  child 0, element: float
pyarrow.Table
event_id: int64
time_group_ids: list<element: int64>
  child 0, element: int64
edge_src_index: list<element: int64>
  child 0, element: int64
edge_dst_index: list<element: int64>
  child 0, element: int64
pred_edge_affinity: list<element: float>
  child 0, element: float
----
event_id: [[0,1,2]]
time_group_ids: [[[0,0,0,0,0,...,2,2,2,2,2],[0,0,0,0,0,...,1,1,1,1,1],[0,0,0,0,0,...,4,4,4,4,4]]]
edge_src_index: [[[0,0,0,0,0,...,57,57,57,57,57],[0,0,0,0,0,...,25,25,25,25,25],[0,0,0,0,0,...,75,75,75,75,75]]]
edge_dst_index: [[[1,2,3,4,5,...,52,53,54,55,56],[1,2,3,4,5,...,20,21,22,23,24],[1,2,3,4,5,...,70,71,72,73,74]]]
pred_edge_affinit